In [12]:
# source https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/

In [30]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /data/home/spushkov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
movies_df2 = pd.read_csv('https://raw.githubusercontent.com/sahildit/IMDB-Movies-Extensive-Dataset-Analysis/master/data1/IMDb%20movies.csv', sep=',', encoding='utf-8')

/data/home/spushkov/.conda/envs/ent_rel_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3369: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
movies_df2.head(3)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0


In [6]:
movies = movies_df2[['imdb_title_id', 'title', 'original_title', 'description', 'genre']]

In [10]:
movies['genres_list'] = movies['genre'].str.split(", ")

/tmp/ipykernel_1128755/393765200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres_list'] = movies['genre'].str.split(", ")


In [11]:
movies.head(3)

,imdb_title_id,title,original_title,description,genre,genres_list
0,tt0000009,Miss Jerry,Miss Jerry,The adventures of a female reporter in the 1890s.,Romance,[Romance]
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,True story of notorious Australian outlaw Ned ...,"Biography, Crime, Drama","[Biography, Crime, Drama]"
2,tt0001892,Den sorte drøm,Den sorte drøm,Two men of high rank are both wooing the beaut...,Drama,[Drama]


In [15]:
# function for text cleaning 
def preproc_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

## Filter out records with no description

In [22]:
movies[movies.isna().any(axis=1)]

,imdb_title_id,title,original_title,description,genre,genres_list
105,tt0008536,Satana likuyushchiy,Satana likuyushchiy,NaN,Drama,[Drama]
158,tt0010219,Hawthorne of the U.S.A.,Hawthorne of the U.S.A.,NaN,"Adventure, Comedy, Romance","[Adventure, Comedy, Romance]"
262,tt0012631,Rojô no reikion,Rojô no reikion,NaN,Drama,[Drama]
450,tt0016377,Soul-Fire,Soul-Fire,NaN,Drama,[Drama]
483,tt0016827,Eve's Leaves,Eve's Leaves,NaN,"Comedy, Romance","[Comedy, Romance]"
...,...,...,...,...,...,...
85566,tt9568486,Mine de rien,Mine de rien,NaN,Comedy,[Comedy]
85684,tt9685652,Greyhound Attack,Greyhound Attack,NaN,"Action, Drama, War","[Action, Drama, War]"
85816,tt9866344,Mon Ami Walid,Mon Ami Walid,NaN,"Comedy, Drama","[Comedy, Drama]"
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,NaN,Drama,[Drama]


In [23]:
movies = movies.dropna(subset=['description'])

In [25]:
movies['preproc_description'] = movies['description'].apply(lambda x: preproc_text(x))

/tmp/ipykernel_1128755/527357254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['preproc_description'] = movies['description'].apply(lambda x: preproc_text(x))


In [26]:
movies

,imdb_title_id,title,original_title,description,genre,genres_list,preproc_description
0,tt0000009,Miss Jerry,Miss Jerry,The adventures of a female reporter in the 1890s.,Romance,[Romance],the adventures of a female reporter in the s
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,True story of notorious Australian outlaw Ned ...,"Biography, Crime, Drama","[Biography, Crime, Drama]",true story of notorious australian outlaw ned ...
2,tt0001892,Den sorte drøm,Den sorte drøm,Two men of high rank are both wooing the beaut...,Drama,[Drama],two men of high rank are both wooing the beaut...
3,tt0002101,Cleopatra,Cleopatra,The fabled queen of Egypt's affair with Roman ...,"Drama, History","[Drama, History]",the fabled queen of egypts affair with roman g...
4,tt0002130,L'Inferno,L'Inferno,Loosely adapted from Dante's Divine Comedy and...,"Adventure, Drama, Fantasy","[Adventure, Drama, Fantasy]",loosely adapted from dantes divine comedy and ...
...,...,...,...,...,...,...,...
85848,tt9905462,Pengalila,Pengalila,An unusual bond between a sixty year old Dalit...,Drama,[Drama],an unusual bond between a sixty year old dalit...
85849,tt9906644,Manoharam,Manoharam,Manoharan is a poster artist struggling to fin...,"Comedy, Drama","[Comedy, Drama]",manoharan is a poster artist struggling to fin...
85850,tt9908390,Le lion,Le lion,A psychiatric hospital patient pretends to be ...,Comedy,[Comedy],a psychiatric hospital patient pretends to be ...
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,A middle-aged veterinary surgeon believes his ...,"Comedy, Drama","[Comedy, Drama]",a middle aged veterinary surgeon believes his ...


## Remove stop-words

In [31]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

movies['preproc_description'] = movies['preproc_description'].apply(lambda x: remove_stopwords(x))

/tmp/ipykernel_1128755/3451000472.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['preproc_description'] = movies['preproc_description'].apply(lambda x: remove_stopwords(x))


In [32]:
movies

,imdb_title_id,title,original_title,description,genre,genres_list,preproc_description
0,tt0000009,Miss Jerry,Miss Jerry,The adventures of a female reporter in the 1890s.,Romance,[Romance],adventures female reporter
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,True story of notorious Australian outlaw Ned ...,"Biography, Crime, Drama","[Biography, Crime, Drama]",true story notorious australian outlaw ned kelly
2,tt0001892,Den sorte drøm,Den sorte drøm,Two men of high rank are both wooing the beaut...,Drama,[Drama],two men high rank wooing beautiful famous eque...
3,tt0002101,Cleopatra,Cleopatra,The fabled queen of Egypt's affair with Roman ...,"Drama, History","[Drama, History]",fabled queen egypts affair roman general marc ...
4,tt0002130,L'Inferno,L'Inferno,Loosely adapted from Dante's Divine Comedy and...,"Adventure, Drama, Fantasy","[Adventure, Drama, Fantasy]",loosely adapted dantes divine comedy inspired ...
...,...,...,...,...,...,...,...
85848,tt9905462,Pengalila,Pengalila,An unusual bond between a sixty year old Dalit...,Drama,[Drama],unusual bond sixty year old dalit worker azhag...
85849,tt9906644,Manoharam,Manoharam,Manoharan is a poster artist struggling to fin...,"Comedy, Drama","[Comedy, Drama]",manoharan poster artist struggling find respec...
85850,tt9908390,Le lion,Le lion,A psychiatric hospital patient pretends to be ...,Comedy,[Comedy],psychiatric hospital patient pretends crazy ch...
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,A middle-aged veterinary surgeon believes his ...,"Comedy, Drama","[Comedy, Drama]",middle aged veterinary surgeon believes wife p...
